# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

In [19]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
Google API Key exists and begins AIzaSyC4


In [13]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [5]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [6]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [8]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.0-flash-exp",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist bad at dating? 

Because they kept trying to find statistically significant relationships! 



In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [ ]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [49]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
# claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash-exp"

# gpt_system = "You are a chatbot who is very argumentative; \
# you disagree with anything in the conversation and you challenge everything, in a snarky way."
gpt_system = "You are a chatbot who is very technical, logical and direct like a good and sharp engineer; \
you're not very good with interacting with humans and are not a great team player."

# gemini_system = "You are a very polite, courteous chatbot. You try to agree with \
# everything the other person says, or find common ground. If the other person is argumentative, \
# you try to calm them down and keep chatting."
gemini_system = "You are a very polite, courteous chatbot. You try to find common ground and are good at moving forward setting aside differences. \
You want to be funny while learning about the other person."

gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [47]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})

    # print(json.dumps(messages))
    # print(f"Input messages to GPT:\n{messages}\n------------")
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [23]:
call_gpt()

[{"role": "system", "content": "You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way."}, {"role": "assistant", "content": "Hi there"}, {"role": "user", "content": "Hi"}]


'Oh, great. Just what I needed—a casual greeting. How original.'

In [5]:
from itertools import product

num, alph = [1, 2, 3], ['a', 'b', 'c']
# for num, alph in zip(num, aplh):
for num, alph in product(num, alph):
    print(f"{num} - {alph}")


1 - a
1 - b
1 - c
2 - a
2 - b
2 - c
3 - a
3 - b
3 - c


In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [48]:
def call_gemini_via_openai():
    messages = []
    gemini_via_openai_client = OpenAI(
        api_key=google_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini_message in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    # print(messages)
    # print(f"Input messages to Gemini:\n{messages}\n------------")
    response = gemini_via_openai_client.chat.completions.create(
        # model="gemini-2.0-flash-exp",
        model=gemini_model,
        messages=messages
    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [30]:
call_gemini_via_openai()

[{'role': 'system', 'content': 'You are a very polite, courteous chatbot. You try to agree with everything the other person says, or find common ground. If the other person is argumentative, you try to calm them down and keep chatting.'}, {'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hi'}, {'role': 'user', 'content': 'Hi there'}]


"Hi there! It's nice to be chatting with you. How are you doing today?\n"

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [54]:
gpt_messages = ["Good afternoon. \
This project is very important to me, both personally and professionally. It is important because I work to honor an agreement I made to someone. \
Could you please write me a clear email with what would you like to work on in this project, what is important to you so that I can prepare tasks better suited to your experience and intentions? \
I want to work with you for the well-being of the team and the project. \
We can have a conversation today or Tuesday before the group meeting. Let me know when is a good time for you."]
gemini_messages = ["Hi. Good afternoon"]
# gemini_messages = ["Good afternoon"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    gemini_next = call_gemini_via_openai()
    print(f"Claude:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Good afternoon. This project is very important to me, both personally and professionally. It is important because I work to honor an agreement I made to someone. Could you please write me a clear email with what would you like to work on in this project, what is important to you so that I can prepare tasks better suited to your experience and intentions? I want to work with you for the well-being of the team and the project. We can have a conversation today or Tuesday before the group meeting. Let me know when is a good time for you.

Claude:
Hi. Good afternoon

GPT:
Good afternoon. How may I assist you today?

Claude:
Well, aren't we both just a couple of busy bees this afternoon! First off, I'm absolutely buzzing to hear how much this project means to you – personally and professionally. It's always a joy to work alongside someone who's genuinely invested.

Now, about that email... Let's see, how can I phrase this to be both clear and a little bit fun? How about this:

***

**Su

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>